##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom layers

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.


In [1]:
import tensorflow as tf

In [2]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


In [3]:
print(tf.config.list_physical_devices('GPU'))

[]


## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.


In [5]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [6]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [7]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.38774234, -0.02461767,  0.43490547,  0.03922373,  0.11500716,
         -0.50492847, -0.31663594,  0.4080674 , -0.5441938 ,  0.383496  ],
        [-0.2885575 , -0.50799763,  0.51968414,  0.29284567, -0.14554757,
          0.09217739, -0.07859552, -0.15251297,  0.10385466,  0.27625853],
        [-0.34293085, -0.04740041, -0.17786139,  0.09215057, -0.31213927,
         -0.13068968, -0.49130678,  0.22157443, -0.31808248,  0.46094114],
        [ 0.451199  ,  0.55882305,  0.6213669 , -0.3633952 ,  0.23620051,
         -0.08101058,  0.38810784, -0.16456676,  0.20450449, -0.13703912],
        [-0.4018246 , -0.4010515 , -0.5423715 ,  0.11078358,  0.09870809,
         -0.5785431 ,  0.18443114, -0.27037036,  0.5258493 , -0.280053  ]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [8]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.38774234, -0.02461767,  0.43490547,  0.03922373,  0.11500716,
         -0.50492847, -0.31663594,  0.4080674 , -0.5441938 ,  0.383496  ],
        [-0.2885575 , -0.50799763,  0.51968414,  0.29284567, -0.14554757,
          0.09217739, -0.07859552, -0.15251297,  0.10385466,  0.27625853],
        [-0.34293085, -0.04740041, -0.17786139,  0.09215057, -0.31213927,
         -0.13068968, -0.49130678,  0.22157443, -0.31808248,  0.46094114],
        [ 0.451199  ,  0.55882305,  0.6213669 , -0.3633952 ,  0.23620051,
         -0.08101058,  0.38810784, -0.16456676,  0.20450449, -0.13703912],
        [-0.4018246 , -0.4010515 , -0.5423715 ,  0.11078358,  0.09870809,
         -0.5785431 ,  0.18443114, -0.27037036,  0.5258493 , -0.280053  ]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

1. `__init__` , where you can do all input-independent initialization
2. `build`, where you know the shapes of the input tensors and can do the rest of the initialization
3. `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [9]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [10]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [11]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


In [12]:
layer.variables

[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.257034  ,  0.36844188,  0.56876534, -0.41178858, -0.2691998 ,
          0.05271965,  0.18430007,  0.57345253,  0.18343318, -0.2312127 ],
        [ 0.6323666 ,  0.3710075 , -0.04683691, -0.05350149,  0.57745403,
         -0.00544453, -0.03945518,  0.07278579,  0.11281228,  0.14792281],
        [ 0.57192487, -0.2897804 , -0.46488762, -0.04659986, -0.2534351 ,
         -0.1660763 ,  0.4979896 , -0.62099195,  0.060265  ,  0.0802387 ],
        [ 0.18601125, -0.01597536,  0.14977932,  0.31150776, -0.5607631 ,
         -0.13654998, -0.61661404,  0.10224664,  0.24024087, -0.19736129],
        [ 0.56941324, -0.48084158, -0.3698505 ,  0.6057307 ,  0.24693173,
          0.2915929 ,  0.28627318, -0.24593633, -0.44425124, -0.10918945]],
       dtype=float32)>]

In [14]:
layer.kernel


<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.257034  ,  0.36844188,  0.56876534, -0.41178858, -0.2691998 ,
         0.05271965,  0.18430007,  0.57345253,  0.18343318, -0.2312127 ],
       [ 0.6323666 ,  0.3710075 , -0.04683691, -0.05350149,  0.57745403,
        -0.00544453, -0.03945518,  0.07278579,  0.11281228,  0.14792281],
       [ 0.57192487, -0.2897804 , -0.46488762, -0.04659986, -0.2534351 ,
        -0.1660763 ,  0.4979896 , -0.62099195,  0.060265  ,  0.0802387 ],
       [ 0.18601125, -0.01597536,  0.14977932,  0.31150776, -0.5607631 ,
        -0.13654998, -0.61661404,  0.10224664,  0.24024087, -0.19736129],
       [ 0.56941324, -0.48084158, -0.3698505 ,  0.6057307 ,  0.24693173,
         0.2915929 ,  0.28627318, -0.24593633, -0.44425124, -0.10918945]],
      dtype=float32)>

In [ ]:
layer.

Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in `tf.keras.layers`, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!

## Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see [Custom Keras layers and models](../../guide/keras/custom_layers_and_models.ipynb) for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [15]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [17]:
print(tf.zeros([1, 2, 3, 3]))

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)


In [18]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [19]:
block.layers

In [20]:
len(block.variables)

18

In [21]:
block.variables

[<tf.Variable 'resnet_identity_block/conv2d/kernel:0' shape=(1, 1, 3, 1) dtype=float32, numpy=
 array([[[[-0.3787186 ],
          [ 0.0594449 ],
          [-0.98577464]]]], dtype=float32)>,
 <tf.Variable 'resnet_identity_block/conv2d/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'resnet_identity_block/batch_normalization/gamma:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>,
 <tf.Variable 'resnet_identity_block/batch_normalization/beta:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'resnet_identity_block/batch_normalization/moving_mean:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'resnet_identity_block/batch_normalization/moving_variance:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>,
 <tf.Variable 'resnet_identity_block/conv2d_1/kernel:0' shape=(1, 1, 1, 2) dtype=float32, numpy=array([[[[1.2794915 , 0.86354744]]]], dtype=float32)>,
 <tf.Variable 'resnet_i

In [22]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [23]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [24]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________

# Next steps

Now you can go back to the previous notebook and adapt the linear regression example to use layers and models to be better structured.